In [12]:
%matplotlib inline
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.finance as mfinance
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import matplotlib.transforms as mtransforms

import pandas as pd
from pandas_datareader import data

In [55]:
from_date = '2016-01-01'
to_date = '2016-06-01'
stock_symbols = ['AAPL','GOOG','AMZN','FB','MSFT']
stocks = {}
for symbol in stock_symbols:
    stocks[symbol] = data.DataReader(symbol,'yahoo',from_date,to_date)

benchmark_symbol = 'SPY'
benchmark = data.DataReader(benchmark_symbol,'yahoo',from_date,to_date)

In [27]:
def plot_chart(stock_data):
    dates = [ mdates.date2num(date) for date in stock_data.index]
    open_prices = stock_data['Open']
    high_prices = stock_data['High']
    low_prices = stock_data['Low']
    close_prices = stock_data['Close']
    volumes = stock_data['Volume'] if 'Volume' in stock_data.columns else None

    chart_data = np.matrix([dates,open_prices,high_prices,low_prices,close_prices]).T.tolist()

    fig = plt.figure(figsize=(15,8))
    ax1 = plt.subplot2grid((5,4),(0,0),rowspan=4,colspan=4)
    mfinance.candlestick_ohlc(ax1,chart_data,colorup='r',colordown='b',alpha=0.7,width=0.7)
    ax1.grid(True)
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    ax1.xaxis_date()
    ax1.autoscale_view()
    plt.ylabel('Price')

    ax2 = plt.subplot2grid((5,4),(4,0),sharex=ax1,rowspan=1,colspan=4)
    ax2.grid(True)

    fig.subplots_adjust(hspace=0)
    plt.setp( plt.gca().get_xticklabels(), rotation=45, horizontalalignment='right')
    plt.setp(ax1.get_xticklabels(),visible=False)
    ax1.set_yticks(ax1.get_yticks()[1:])
    return ax1,ax2

In [65]:
stock_data = stocks['AAPL']

class GDStrategy(object):
    def __init__(self):
        pass
    def is_enter(self,data,**kwarg):
        price = data['Close']
        long_ma = kwarg['long_ma']
        short_ma = kwarg['short_ma']
        short_ma_data = pd.rolling_mean(price,window=short_ma).dropna()
        long_ma_data = pd.rolling_mean(price,window=long_ma).dropna()
        if short_ma_data is None or len(short_ma_data) < 2:
            return False
        if long_ma_data is None or len(long_ma_data) < 2:
            return False
        if short_ma_data.values[-2] < long_ma_data.values[-2] and short_ma_data.values[-1] > long_ma_data.values[-1]:
            return True
        return False
    def is_exit(self,data,**kwarg):
        price = data['Close']
        long_ma = kwarg['long_ma']
        short_ma = kwarg['short_ma']
        short_ma_data = pd.rolling_mean(price,window=short_ma)
        long_ma_data = pd.rolling_mean(price,window=long_ma)
        if short_ma_data is None or len(short_ma_data) < 2:
            return False
        if long_ma_data is None or len(long_ma_data) < 2:
            return False
        if short_ma_data.values[-2] > long_ma_data.values[-2] and short_ma_data.values[-1] < long_ma_data.values[-1]:
            return True
        return False
    
def evaluate_strategy(data,strategy,**kwarg):
    buying_price = 0.
    profit = 0.
    for i in range(1,len(data)):
        if strategy.is_enter(data[:i],**kwarg):
            print 'enter : ',data[:i].index[-1],data[:i]['Close'][-1]
            buying_price = data[:i]['Close'][-1]
        elif strategy.is_exit(data[:i],**kwarg):
            print 'exit : ',data[:i].index[-1],data[:i]['Close'][-1]
            profit += data[:i]['Close'][-1]/buying_price - 1.
    return profit
        
def optimize_strategy(data,strategy,**kwarg):
    print kwarg

gds = GDStrategy()
# print evaluate_strategy(stock_data,gds,long_ma=20,short_ma=5)
optimize_strategy(stock_data,gds,long_mas=[10,20],short_mas=[5,9])

# long_ma = pd.rolling_mean(stock_data['Close'],window=20).dropna()
# short_ma = pd.rolling_mean(stock_data['Close'],window=5).dropna()
# ax1,ax2 = plot_chart(stock_data)
# ax1.plot(long_ma.index.values,long_ma.values)
# ax1.plot(short_ma.index.values,short_ma.values)

{'long_mas': [10, 20], 'short_mas': [5, 9]}
